In [2]:
#Connection to MYSQL(localhost)

In [1]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

In [8]:
def connector(host, db, root, password, ssq, columns):
    try:
        connection = mysql.connector.connect(host=host,
                                       database=db,
                                       user=root,
                                       password=password)
        if connection.is_connected():
            db_Info = connection.get_server_info()
            print('Connection to MySQL Server', db_Info)
            cursor = connection.cursor()
            cursor.execute('SELECT database();')
            record = cursor.fetchone()
            print("You are connected to database: ", record)
            
            sql_select_Query = ssq
            cursor = connection.cursor(dictionary=True)
            cursor.execute(sql_select_Query)
            records = cursor.fetchall()
            
            df = pd.DataFrame(records, columns=columns)


    except Erro as e:
        print('Error connection to MySQL', e)
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print('MySQL connection is closed')
    return df

In [9]:
col = ['city_id','city','country_id','last_update']
ssq= "SELECT * FROM city ORDER BY city_id"
host='localhost'
db='sakila'
root='root'
password='pass'
df = connector(host, db, root,password,ssq,col)

Connection to MySQL Server 8.0.29
You are connected to database:  ('sakila',)
MySQL connection is closed


In [10]:
df

,city_id,city,country_id,last_update
0,1,A Coruña (La Coruña),87,2006-02-15 04:45:25
1,2,Abha,82,2006-02-15 04:45:25
2,3,Abu Dhabi,101,2006-02-15 04:45:25
3,4,Acuña,60,2006-02-15 04:45:25
4,5,Adana,97,2006-02-15 04:45:25
...,...,...,...,...
595,596,Zaria,69,2006-02-15 04:45:25
596,597,Zeleznogorsk,80,2006-02-15 04:45:25
597,598,Zhezqazghan,51,2006-02-15 04:45:25
598,599,Zhoushan,23,2006-02-15 04:45:25


In [ ]:
#Exporting DataFrame to Google cloud platform backet

In [32]:
import google.cloud
from google.cloud import storage

In [36]:
def bucket_gcp(myjsonfilekey, bucket):
    client = storage.Client.from_service_account_json(myjsonfilekey)
    bucket = client.get_bucket(bucket)
    bucket.blob('upload_address/address.csv').upload_from_string(df.to_csv(), 'text/csv')

In [37]:
myjsonfilekey = 'yourkey.json'
bucket = 'etl_backet'
bucket_gcp(myjsonfilekey, bucket)

In [ ]:
#Here we reached backets into GCP but it is comfortable only if you want to store a file

In [ ]:
################################################################################################################

In [38]:
#Exporting data frame to Google Cloud Platform Big Query

In [11]:
from google.cloud import storage
from google.oauth2 import service_account
import pandas_gbq

In [12]:

projectid = 'elt-to-bigquery'
myjsonfilekey = 'yourkey.json'
destination = "Shark_set_bq_2022.My_bq_transfer_sql"

In [13]:
def con_to_big_query(myjsonfilekey, projectid, destination, tableschema):
    credentials_ = service_account.Credentials.from_service_account_file(myjsonfilekey)
    df.to_gbq(project_id=projectid,
              destination_table=destination,
              credentials=credentials_,
             chunksize=None,
             if_exists='replace',
            table_schema= tableschema
             )

In [14]:
projectid = 'elt-to-bigquery'
myjsonfilekey = 'yourkey.json'
destination = "Shark_set_bq_2022.My_bq_transfer_sql"
table_schema = [{'name':'city_id', 'type':'INTEGER'},
                          {'name':'city', 'type':'STRING'},
                          {'name':'country_id', 'type':'INTEGER'},
                        {'name':'last_update', 'type':'DATETIME'} ]

con_to_big_query(myjsonfilekey,projectid, destination, table_schema)

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 1543.16it/s]


In [ ]:
#Here we go, our table has been downloaded without any problems. This solution could be usefull for those who tried libraries
#like google.cloud bigquery but it dosen't work because of version error same as i have. Pandas_gbq gives much more simplier solution